In [6]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from ast import literal_eval

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [25]:
THEMES = [5, 6, 26, 33, 139, 163, 232, 313, 339, 350, 406, 409, 555, 589,
          597, 634, 660, 695, 729, 766, 773, 793, 800, 810, 852, 895, 951, 975]
TRAIN_DATA_PATH = '../train_small.csv'
TEST_DATA_PATH = '../test_small.csv'
VALIDATION_DATA_PATH = '../validation_small.csv'

In [29]:
def get_data(path, preds=None, key=None):
    data = pd.read_csv(path)
    data = data.rename(columns={ 'pages': 'page'})
    data.body = data.body.str.strip('{}"')
    data = groupby_process(data)
    data.themes = data.themes.apply(lambda x: literal_eval(x))
    return data

In [30]:
def groupby_process(df):
    new_df = df.sort_values(['process_id', 'page'])
    new_df = new_df.groupby(
                ['process_id', 'themes'],
                group_keys=False
            ).apply(lambda x: x.body.str.cat(sep=' ')).reset_index()
    new_df = new_df.rename(index=str, columns={0: "body"})
    return new_df

In [31]:
train_data = get_data(TRAIN_DATA_PATH)
test_data = get_data(TEST_DATA_PATH)
validation_data = get_data(VALIDATION_DATA_PATH)

In [32]:
train_data.themes = train_data.themes.apply(lambda x: list(set(sorted([i if i in THEMES else 0 for i in x]))))
test_data.themes = test_data.themes.apply(lambda x: list(set(sorted([i if i in THEMES else 0 for i in x]))))
validation_data.themes = validation_data.themes.apply(lambda x: list(set(sorted([i if i in THEMES else 0 for i in x]))))

In [33]:
len(train_data), len(validation_data), len(test_data)

(2743, 1893, 1874)

In [34]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(sentence.split(" "))

train_words = list(sent_to_words(train_data.body.tolist()))


print(train_words[:1])

[['tribunal', 'justiça', 'estado', 'bahia', 'poder', 'judiciário', 'salvador', 'juizado', 'cível', 'defesa', 'consumidor', 'universo', 'matutino', 'projudi', 'acm', 'campus', 'universidade', 'salgado', 'oliveira', 'universo', 'pituba', 'salvador', 'processo', 'parte', 'autora', 'nailton', 'lantyer', 'cordeiro', 'araujo', 'parte', 'atlantico', 'fundo', 'investimento', 'direitos', 'creditorios', 'despacho', 'vistos', 'etc', 'indefiro', 'requerido', 'pela', 'parte', 'demandante', 'aguarde', 'a', 'sessão', 'conciliação', 'designada', 'salvador', 'de', 'julho', 'paulo', 'alberto', 'nunes', 'chenaud', 'juiz', 'direito', 'documento', 'assinado', 'eletronicamente', 'excelentíssimo', 'senhor', 'doutor', 'juiz', 'direito', 'juizado', 'cível', 'defesa', 'consumidor', 'universo', 'comarca', 'salvador', 'autos', 'processo', 'º', 'atlântico', 'fundo', 'investimento', 'direitos', 'creditórios', 'não', 'padronizados', 'nova', 'denominação', 'crdg', 'fundo', 'investimento', 'direitos', 'creditórios', '

In [35]:
len(train_words)

2743

In [37]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(train_words, min_count=50, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[train_words], min_count=5, threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[train_words[0]]])

['tribunal', 'justiça', 'estado', 'bahia', 'poder', 'judiciário', 'salvador', 'juizado', 'cível', 'defesa_consumidor', 'universo', 'matutino_projudi', 'acm', 'campus', 'universidade', 'salgado', 'oliveira', 'universo', 'pituba', 'salvador', 'processo', 'parte', 'autora', 'nailton_lantyer', 'cordeiro', 'araujo', 'parte', 'atlantico_fundo', 'investimento', 'direitos', 'creditorios', 'despacho', 'vistos_etc', 'indefiro', 'requerido', 'pela', 'parte', 'demandante', 'aguarde', 'a', 'sessão', 'conciliação', 'designada', 'salvador', 'de', 'julho', 'paulo', 'alberto', 'nunes', 'chenaud', 'juiz', 'direito', 'documento', 'assinado', 'eletronicamente', 'excelentíssimo_senhor_doutor', 'juiz', 'direito', 'juizado', 'cível', 'defesa_consumidor', 'universo', 'comarca', 'salvador', 'autos', 'processo', 'º', 'atlântico_fundo', 'investimento', 'direitos_creditórios', 'não', 'padronizados', 'nova', 'denominação', 'crdg', 'fundo_investimento', 'direitos_creditórios', 'não', 'padronizados', 'empresa', 'ins

In [38]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [39]:
train_words = make_bigrams(train_words)
train_words = make_trigrams(train_words)

In [40]:
# Create Dictionary
id2word = corpora.Dictionary(train_words)

In [42]:
def save_dic(dic, filename="dic"):
    with open(filename, "wb") as handle:
        dic.save(handle)

In [43]:
save_dic(id2word, "small_dict")

In [45]:
len(id2word.dfs), id2word.dfs

(221049,
 {2415: 2740,
  1409: 2072,
  949: 2050,
  270: 188,
  1810: 1240,
  1378: 1924,
  2187: 139,
  1380: 1165,
  625: 1187,
  669: 97,
  2436: 159,
  1502: 2,
  58: 3,
  326: 12,
  2435: 109,
  2182: 42,
  1684: 1104,
  1799: 1,
  1916: 2707,
  1733: 1827,
  258: 1169,
  1578: 1,
  574: 117,
  196: 312,
  235: 2,
  1349: 69,
  788: 1042,
  586: 2,
  727: 2268,
  2502: 261,
  1255: 249,
  2119: 708,
  1762: 1635,
  687: 574,
  113: 129,
  31: 2453,
  2242: 1056,
  468: 198,
  720: 131,
  649: 2741,
  1393: 1204,
  1752: 1682,
  116: 514,
  1641: 375,
  381: 1,
  1379: 1210,
  787: 2347,
  813: 2552,
  218: 2416,
  865: 543,
  987: 841,
  425: 232,
  263: 2699,
  2527: 1089,
  237: 2,
  789: 5,
  1644: 1945,
  1721: 20,
  1635: 1083,
  698: 107,
  585: 1,
  1130: 16,
  881: 412,
  1309: 447,
  400: 529,
  2262: 2553,
  424: 2589,
  2202: 997,
  265: 184,
  302: 12,
  1040: 316,
  1445: 667,
  145: 737,
  2325: 1864,
  1819: 2369,
  2245: 1187,
  94: 532,
  2251: 15,
  1630: 2496,
 

In [46]:
id2word.filter_extremes(no_below=50, no_above=0.5)

In [47]:
len(id2word.dfs), id2word.dfs

(11758,
 {191: 188,
  1218: 1240,
  1488: 139,
  922: 1165,
  423: 1187,
  451: 97,
  1653: 159,
  1652: 109,
  1130: 1104,
  181: 1169,
  393: 117,
  131: 312,
  906: 69,
  532: 1042,
  1705: 261,
  842: 249,
  1436: 708,
  468: 574,
  69: 129,
  1525: 1056,
  314: 198,
  490: 131,
  933: 1204,
  71: 514,
  1101: 375,
  921: 1210,
  587: 543,
  670: 841,
  286: 232,
  1721: 1089,
  1095: 1083,
  479: 107,
  596: 412,
  877: 447,
  272: 529,
  1497: 997,
  186: 184,
  699: 316,
  973: 667,
  95: 737,
  1527: 1187,
  54: 532,
  189: 1226,
  626: 1203,
  1320: 898,
  1674: 514,
  671: 697,
  838: 524,
  1364: 1202,
  741: 923,
  758: 821,
  12: 457,
  961: 1328,
  899: 1041,
  876: 1019,
  1379: 943,
  568: 1148,
  364: 198,
  1369: 1183,
  97: 523,
  13: 169,
  1164: 953,
  17: 828,
  1388: 1202,
  867: 861,
  1265: 318,
  1524: 1166,
  32: 87,
  803: 731,
  1501: 1203,
  900: 1033,
  498: 1324,
  482: 177,
  1431: 1193,
  1611: 1067,
  1341: 63,
  1695: 78,
  546: 611,
  1128: 896,
  9

In [62]:
[(id2word[x], y) for (x, y) in sorted(id2word.dfs.items(), key=lambda x: x[1], reverse=True)]

[('automaticamente', 1369),
 ('atual', 1367),
 ('advogado', 1364),
 ('sua', 1363),
 ('dada', 1363),
 ('regimento_interno', 1362),
 ('termo_remessa_externa', 1362),
 ('faço_remessa_destes', 1362),
 ('determino_devolução_destes', 1361),
 ('contra', 1360),
 ('institui_infra_estrutura_chaves', 1359),
 ('peças_processuais', 1359),
 ('instituto', 1358),
 ('outubro', 1356),
 ('nesta', 1356),
 ('públicas_brasileira_icp_brasil', 1355),
 ('caput', 1350),
 ('prazo', 1350),
 ('anterior', 1343),
 ('ministra_cármen_lúcia', 1343),
 ('concessão', 1330),
 ('lei', 1328),
 ('base', 1326),
 ('seja', 1325),
 ('devido', 1325),
 ('deste', 1324),
 ('setembro', 1324),
 ('considerando', 1323),
 ('inc', 1323),
 ('aposentadoria', 1323),
 ('legais', 1319),
 ('juizados', 1319),
 ('cpc', 1318),
 ('abril', 1318),
 ('interposto', 1315),
 ('maio', 1315),
 ('ciência', 1312),
 ('caso', 1310),
 ('procedimento', 1308),
 ('remessa', 1306),
 ('após', 1303),
 ('baixa', 1303),
 ('março', 1299),
 ('esse', 1295),
 ('seguro', 129

In [65]:
save_dic(id2word, "small_dict")

In [66]:
train_corpus = [id2word.doc2bow(text) for text in train_words]

In [68]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=train_corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=42,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [69]:
pprint(lda_model.print_topics())

[(0,
  '0.018*"lei" + 0.011*"aposentadoria" + 0.009*"constituição" + '
  '0.009*"servidor" + 0.009*"servidores" + 0.008*"fazenda" + 0.008*"pública" + '
  '0.008*"remuneração" + 0.007*"anos" + 0.006*"proventos"'),
 (1,
  '0.019*"tempo" + 0.016*"aposentadoria" + 0.010*"atividade" + 0.010*"inss" + '
  '0.010*"benefício" + 0.009*"serviço" + 0.007*"concessão" + 0.007*"segurado" '
  '+ 0.007*"previdenciário" + 0.006*"procuradoria"'),
 (2,
  '0.052*"saúde" + 0.041*"pública" + 0.030*"r" + 0.025*"c" + 0.024*"d" + '
  '0.024*"agente" + 0.022*"u" + 0.020*"i" + 0.015*"t" + 0.012*"fls"'),
 (3,
  '0.038*"instituto" + 0.037*"seguro" + 0.034*"advogado" + 0.027*"previd" + '
  '0.024*"maria" + 0.022*"juiz" + 0.021*"oliveira" + 0.020*"mpf" + '
  '0.019*"santos" + 0.017*"dpu"'),
 (4,
  '0.005*"caso" + 0.005*"uma" + 0.005*"sua" + 0.004*"ação" + 0.004*"seja" + '
  '0.004*"união" + 0.004*"prazo" + 0.004*"assim" + 0.004*"poder" + '
  '0.004*"rio"'),
 (5,
  '0.026*"fazenda" + 0.022*"renda" + 0.021*"imposto" + 

In [70]:
lda_model.save("lda_small_first")

/home/isis/.env/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/home/isis/.env/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [74]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(train_corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=train_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.532725195321219


OSError: [Errno 12] Cannot allocate memory

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis